#Initialization:

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from numpy import load
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path1 = '/content/drive/MyDrive/VU/project_flatlands/flatland_train.npz'

Mounted at /content/drive


In [3]:
data = np.load(path1)
X = data['X']
y = data['y']

y[y != 0] -= 2    # Correct labels so that triangle is mapped to class 1
X = X / 255.      # Scale down to range [0, 1]

In [4]:
print(X.shape)
print(y.shape)

(10000, 50, 50)
(10000,)


[2. 4. 3. ... 4. 4. 3.]


In [5]:
min_value = y.min()
max_value = y.max()
print(min_value)
print(max_value)

0.0
4.0


#Data Preparation:

In [6]:
class FlatLensDataset(Dataset):
    def __init__(self, X, y):
        """
        Custom dataset initialization.
        """
        self.X = X
        self.y = y

    def __len__(self):
        """
        Return the total number of samples in the dataset.
        """
        return len(self.X)

    def __getitem__(self, idx):
        """
        Generates one sample of data.
        """
        # Loading and getting the image
        image = self.X[idx]
        # Add a channel dimension (1, 50, 50), for grayscale images
        image = image.reshape(1, 50, 50)
        image = torch.tensor(image, dtype=torch.float32)

        # Get label
        label = self.y[idx]
        label = torch.tensor(label, dtype=torch.long)

        return image, label

In [7]:
def split_dataset(X, y, train_ratio=0.8):
    """
    Splits the dataset into training and testing sets based on the specified ratio.

    :param X: The input features.
    :param y: The target labels.
    :param train_ratio: Ratio of the training set size to the total dataset size.
    :return: Two datasets, one for training and one for testing.
    """
    # Create the full dataset
    full_dataset = FlatLensDataset(X, y)

    # Calculate the size of training and testing datasets
    total_size = len(full_dataset)
    train_size = int(total_size * train_ratio)
    test_size = total_size - train_size

    # Split the dataset
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

    return train_dataset, test_dataset

# Usage example
train_ratio = 0.8  # 80% training, 20% testing
train_dataset, test_dataset = split_dataset(X, y, train_ratio)

# Now create DataLoaders for both datasets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model:

In [8]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1) # for grayscale images
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 12 * 12, 128) # Adjust the size
        self.fc2 = nn.Linear(128, 5) # 5 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 12 * 12) # Flatten the output for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        if not self.training:
            # In evaluation mode, return the predicted class directly
            return torch.argmax(x, dim=1)
        return x  # In training mode, return logits


#Train loop:

In [19]:
### Initialize model"

# Check if CUDA is available and set the device to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Move the model to the specified device
model = SimpleCNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

Using device: cuda


In [20]:
num_epochs = 25  # Set the number of epochs
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/25], Loss: 1.1916
Epoch [2/25], Loss: 1.0745
Epoch [3/25], Loss: 0.9985
Epoch [4/25], Loss: 0.7813
Epoch [5/25], Loss: 0.8154
Epoch [6/25], Loss: 0.6428
Epoch [7/25], Loss: 0.5147
Epoch [8/25], Loss: 0.6266
Epoch [9/25], Loss: 0.3904
Epoch [10/25], Loss: 0.4598
Epoch [11/25], Loss: 0.2070
Epoch [12/25], Loss: 0.2149
Epoch [13/25], Loss: 0.1602
Epoch [14/25], Loss: 0.2904
Epoch [15/25], Loss: 0.1329
Epoch [16/25], Loss: 0.1257
Epoch [17/25], Loss: 0.2970
Epoch [18/25], Loss: 0.1030
Epoch [19/25], Loss: 0.1473
Epoch [20/25], Loss: 0.1554
Epoch [21/25], Loss: 0.1221
Epoch [22/25], Loss: 0.1062
Epoch [23/25], Loss: 0.2004
Epoch [24/25], Loss: 0.0663
Epoch [25/25], Loss: 0.1075


In [21]:
# Save the entire model
torch.save(model, 'model.pth')

#Test model:

In [22]:
# Load the entire model
model = torch.load('model.pth')

# Set the model to evaluation mode
model.eval()

# Initialize variables to track test accuracy and total number of test samples
correct = 0
total = 0

# Disable gradient computation since we are in evaluation mode
with torch.no_grad():
    for images, labels in test_loader:
        # Move the data to the device
        images, labels = images.to(device), labels.to(device)

        # Forward pass to get outputs
        outputs = model(images)

        # Get predictions
        predicted = outputs.data

        # Total number of labels
        total += labels.size(0)

        # Total correct predictions
        correct += (predicted == labels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy}%')

Accuracy of the model on the test images: 93.65%
